In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET
import gpxpy
import gpxpy.gpx
import math
import matplotlib.pyplot as plt
import datetime as dt
import re
import matplotlib.dates as mdates
import japanize_matplotlib
from pytz import timezone
from shapely.geometry import Point
import json
import geojson
import os
import glob
from geojson import Point, Feature, FeatureCollection, dump, LineString, Polygon
import exifread

plt.style.use("ggplot")
date = dt.datetime(2024, 12, 14)
days = 1 #日数
mountain = "鋸山・大岳山"

def main():
    track_to_csv(mountain, date)
    mk_altitude_time_map(mountain, date, days)
    mk_altitude_distance_map(mountain, date, days)
    mk_distance_time_map(mountain, date, days)
    #make_track_map(mountain, date, days)
    #pic_to_geojson()
    #csv_to_geojson()


## 2点間の距離の計算
def cal_distance(lon1, lat1, lon2, lat2):
    R = 6371.0  # 地球の半径 (km)
    dlat = math.radians(lat2 - lat1)  # 緯度の差をラジアンに変換
    dlon = math.radians(lon2 - lon1)  # 経度の差をラジアンに変換
    # ハヴァサイン公式の計算
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R*c  #距離を返す


## トラックをCSVに変換
def track_to_csv(mountain, date):
    ifile = "./10_data/{yyyy:04d}{mm:02d}{dd:02d}_{mountain}.gpx".format(yyyy=date.year, mm=date.month, dd=date.day, mountain=mountain)
    # タイムゾーン
    dt_tz = timezone('Asia/Tokyo')
    # 日時文字列形式
    dt_fmt = '%Y-%m-%d %H:%M:%S'
    # GPXファイルの読み込み
    gpx_file_r = open(ifile, 'r')
    # GPXファイルのパース
    gpx = gpxpy.parse(gpx_file_r)
    print(ifile)
    counts = 0
    for track in gpx.tracks:
        for segment in track.segments:
            points = segment.points
            #print(points.longitude)
            N = len(points)
            lons = np.zeros(N); lats = np.zeros_like(lons); alts = np.zeros_like(lons); dis = np.zeros_like(lons); times = ["a"]*N
            for i in range(N):
                lons[i] = points[i].longitude
                lats[i] = points[i].latitude
                alts[i] = points[i].elevation
                times[i] = points[i].time.astimezone(dt_tz).strftime(dt_fmt)

            for i in range(1, N):
                dis[i] = dis[i-1] + cal_distance(lons[i-1], lats[i-1], lons[i], lats[i])
    
            date = points[0].time.astimezone(dt_tz)
            ofile = "./10_data/{yyyy:04d}{mm:02d}{dd:02d}_{mountain}_track.csv".format(yyyy=date.year, mm=date.month, dd=date.day, mountain=mountain)
            df = pd.DataFrame()
            df["time"] = times; df["longitude"] = lons; df["latitude"] = lats; df["altitude"] = alts; df["distance"] = dis
            df.to_csv(ofile, index=False, sep=" ")
            print("ofile", ofile)
        counts += 1


## 高度-時間断面図
def mk_altitude_time_map(mountain, date, days=1):
    for i in range(days):
        dt_buf = date + dt.timedelta(days=i)
        ifile = "./10_data/{year:04d}{month:02d}{day:02d}_{mountain}_track.csv".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)
        ofile = "./20_map/{year:04d}{month:02d}{day:02d}_{mountain}_altitude-time_map.png".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)

        df = pd.read_csv(ifile, sep=" ")
        times = pd.to_datetime(df["time"])
        ts = pd.to_datetime(df["time"]).values
        alts = df["altitude"].values

        fig, ax = plt.subplots(figsize=(16, 9)) 

        ax.plot(ts, alts)
        ax.set_title("{year:04d}/{month:02d}/{day:02d}, {mountain}".format(mountain=mountain, year=dt_buf.year, month=dt_buf.month, day=dt_buf.day), fontsize=35, loc="left")
        ax.set_xlabel("time", fontsize=30)
        ax.set_ylabel("Altitude [m]", fontsize=30)
        ax.tick_params(labelsize=25)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
        plt.savefig(ofile)
        plt.close()
        print("ifile: ", ifile)
        print("ofile: ", ofile)


## 高度-距離断面図
def mk_altitude_distance_map(mountain, date, days=1):
    for i in range(days):
        dt_buf = date + dt.timedelta(days=i)
        ifile = "./10_data/{year:04d}{month:02d}{day:02d}_{mountain}_track.csv".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)
        ofile = "./20_map/{year:04d}{month:02d}{day:02d}_{mountain}_altitude-distance_map.png".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)

        df = pd.read_csv(ifile, sep=" ")
        xs = df["distance"].values
        alts = df["altitude"].values

        fig, ax = plt.subplots(figsize=(16, 9)) 

        ax.plot(xs, alts)
        ax.set_title("{year:04d}/{month:02d}/{day:02d}, {mountain}".format(mountain=mountain, year=dt_buf.year, month=dt_buf.month, day=dt_buf.day), fontsize=35, loc="left")
        ax.set_xlabel("Distance [km]", fontsize=30)
        ax.set_ylabel("Altitude [m]", fontsize=30)
        ax.tick_params(labelsize=25)
        plt.savefig(ofile)
        plt.close()

        print("ifile: ", ifile)
        print("ofile: ", ofile)


## 時間-距離断面図
def mk_distance_time_map(mountain, date, days=1):
    for i in range(days):
        dt_buf = date + dt.timedelta(days=i)
        ifile = "./10_data/{year:04d}{month:02d}{day:02d}_{mountain}_track.csv".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)
        ofile = "./20_map/{year:04d}{month:02d}{day:02d}_{mountain}_distance-time_map.png".format(year=dt_buf.year, month=dt_buf.month, day=dt_buf.day, mountain=mountain)

        df = pd.read_csv(ifile, sep=" ")
        xs = df["distance"].values
        ts = pd.to_datetime(df["time"]).values

        fig, ax = plt.subplots(figsize=(16, 9)) 

        ax.plot(ts, xs)
        ax.set_title("{year:04d}/{month:02d}/{day:02d}, {mountain}".format(mountain=mountain, year=dt_buf.year, month=dt_buf.month, day=dt_buf.day), fontsize=35, loc="left")
        ax.set_ylabel("Distance [km]", fontsize=30)
        ax.set_xlabel("Time", fontsize=30)
        ax.tick_params(labelsize=25)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
        plt.savefig(ofile)
        plt.close()

        print("ifile: ", ifile)
        print("ofile: ", ofile)


if __name__ == "__main__":
    main()

./10_data/20241214_鋸山・大岳山.gpx
ofile ./10_data/20241214_鋸山・大岳山_track.csv
ifile:  ./10_data/20241214_鋸山・大岳山_track.csv
ofile:  ./20_map/20241214_鋸山・大岳山_altitude-time_map.png
ifile:  ./10_data/20241214_鋸山・大岳山_track.csv
ofile:  ./20_map/20241214_鋸山・大岳山_altitude-distance_map.png
ifile:  ./10_data/20241214_鋸山・大岳山_track.csv
ofile:  ./20_map/20241214_鋸山・大岳山_distance-time_map.png


In [3]:
import re
import glob
import pandas as pd
import datetime as dt
from geojson import Point, Feature, FeatureCollection, dump, LineString, Polygon
import numpy as np
import json

def main():
    idir = "./10_data/"
    files = glob.glob(f"{idir}*.csv")
    features = []
    ofile = "./10_data/track.geojson"
    for file in files:
        print(file)

        pattern_date = re.compile("[0-9]{8}")
        pattern_mountain = re.compile(r"[0-9]{8}_(.+)_track")
        res = pattern_date.search(file).group()
        date = dt.datetime.strptime(res, '%Y%m%d')
        res = pattern_mountain.search(file)
        mountain = res.group(1)
        print(date, mountain)

        
        idf = pd.read_csv(file, sep=" ")
        xy = [xy for xy in zip(idf["longitude"], idf["latitude"])]
        line = LineString(xy)
        start_time = idf["time"][0]
        end_time = idf["time"].values[-1]
        max_altitude = np.max(idf["altitude"])
        min_altitude = np.min(idf["altitude"])
        total_distance = idf["distance"].values[-1]

        feature = {
            "type": "Feature",
            "properties": {
                "mountain": mountain,
                "date": f"{date.year:04d}{date.month:02d}{date.day:02d}",
                "start_time": str(start_time),
                "end_time": str(end_time),
                "max_altitude": str(max_altitude),
                "min_altitude": str(min_altitude),
                "total_distance": str(total_distance),
                "alt_time_map": f"./20_map/{date.year:04d}{date.month:02d}{date.day:02d}_{mountain}_altitude-time_map.png",
                "alt_distance_map": f"./20_map/{date.year:04d}{date.month:02d}{date.day:02d}_{mountain}_altitude-distance_map.png",
                "distance_time_map": f"./20_map/{date.year:04d}{date.month:02d}{date.day:02d}_{mountain}_distance-time_map.png",
            },
            "geometry": {
                "type": "LineString",
                "coordinates": xy,
            },
        }

        features.append(feature)

    # GeoJSONファイルを書き込む
    with open(ofile, "w", encoding='utf_8') as f:
        json.dump({"type":"FeatureCollection", "features": features}, f, indent = 2, ensure_ascii=False)
        print(f"ofile: {ofile}")


if __name__ == "__main__":
    main()

./10_data/20241214_鋸山・大岳山_track.csv
2024-12-14 00:00:00 鋸山・大岳山
ofile: ./10_data/track.geojson


In [7]:

import numpy as np
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET
import gpxpy
import gpxpy.gpx
import math
import matplotlib.pyplot as plt
import datetime as dt
import re
import matplotlib.dates as mdates
import japanize_matplotlib
from pytz import timezone
from shapely.geometry import Point
import json
import geojson
import os
import glob
from geojson import Point, Feature, FeatureCollection, dump, LineString, Polygon
import exifread
import sys

def main():
    pic_to_geojson()

def read_exif_from_image(image_path):
    """画像ファイルからEXIFデータを読み込む"""
    try:
        with open(image_path, "rb") as f:
            exif_data = exifread.process_file(f)
        return exif_data
    except FileNotFoundError:
        print(f"ファイルが見つかりません: {image_path}")
        return None
    except IOError:
        print(f"ファイルの読み込みに失敗しました: {image_path}")
        return None

        
def dms_to_decimal(dms):
    """度分秒（DMS）リストから10進数の座標に変換する関数"""
    degrees, minutes, seconds = dms
    return degrees + (minutes / 60) + (seconds / 3600)


def get_coordinates(exif_data):
    """EXIFタグから緯度と経度を抽出してタプルで返す"""
    try:
        pre_latitude = eval(exif_data["GPS GPSLatitude"].printable)
        pre_longitude = eval(exif_data["GPS GPSLongitude"].printable)
        datetime = exif_data['EXIF DateTimeDigitized'].printable
        alt = eval(exif_data['GPS GPSAltitude'].printable)
        latitude = dms_to_decimal(pre_latitude)
        longitude = dms_to_decimal(pre_longitude)
        return (datetime, latitude, longitude, alt)
    except KeyError:
        print("必要なGPS情報が含まれていません。")
        return None


def pic_to_geojson():
    ofile = "./10_data/img_info.geojson"
    pictures = glob.glob("./img/*.jpg")
    n = len(pictures)
    datetimes = ["date"]*n
    lats = np.zeros(n)
    lons = np.zeros(n)
    alts = np.zeros(n)

    for i in range(n):
        image_path = pictures[i]
        print(f"ifile: {image_path}")
        exif_data = read_exif_from_image(image_path)
        if exif_data:
            result = get_coordinates(exif_data)
            if result:
                #print("日時，緯度，経度，標高:", result)
                datetimes[i], lats[i], lons[i], alts[i] = result
                #datetime, lat , lon, alt = result
            else:
                print("GPS情報の取得に失敗しました。")
                sys.exit(1)
        else:
            print("EXIFデータの読み込みに失敗しました。")
            sys.exit(1)

    geometry = [  Point(xy) for xy in zip(lons, lats)]
    df = pd.DataFrame()
    df["datetime"] = datetimes
    df["altitude"] = alts
    df["name"] = pictures

    geo_df = gpd.GeoDataFrame(df, geometry=geometry)
    geo_df.to_file(ofile, driver='GeoJSON', encoding='utf-8')
    print("ofile:", ofile)


if __name__ == "__main__":
    main()

ifile: ./img/PXL_20241213_214811296.jpg
ifile: ./img/PXL_20241213_215843668.jpg
ifile: ./img/PXL_20241213_220510367.jpg
ifile: ./img/PXL_20241213_224921183.jpg
ifile: ./img/PXL_20241213_232413981.jpg
ifile: ./img/PXL_20241213_235808838.jpg
ifile: ./img/PXL_20241213_235857273.PANO.jpg
ifile: ./img/PXL_20241214_002342949.jpg
ifile: ./img/PXL_20241214_003051708.jpg
ifile: ./img/PXL_20241214_011738676.jpg
ifile: ./img/PXL_20241214_011953728.PANO.jpg
ifile: ./img/PXL_20241214_012110759.jpg
ifile: ./img/PXL_20241214_030441851.jpg
ifile: ./img/PXL_20241214_034455841.jpg
ifile: ./img/PXL_20241214_035552699.jpg
ifile: ./img/PXL_20241214_041259810.jpg
ifile: ./img/PXL_20241214_042941420.jpg
ifile: ./img/PXL_20241214_043309864.jpg
ifile: ./img/PXL_20241214_043324599.jpg
ifile: ./img/PXL_20241214_051406313.jpg
ifile: ./img/PXL_20241214_051454686.jpg
ofile: ./10_data/img_info.geojson


/home/mineta/anaconda3/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [88]:
def csv_to_geojson():
    ifiles = glob.glob("./01_data/*_track.csv")
    all_df = pd.DataFrame()
    ofile = "./10_data/track.geojson"
    total_distance = 0
    for ifile in ifiles:
        each_df = pd.read_csv(ifile, sep=" ")
        #total_distance  += each_df["distance"][-1]
        all_df = pd.concat([all_df, each_df])

    xy = [xy for xy in zip(all_df["longitude"], all_df["latitude"])]
    line = LineString(xy)
    start_time = all_df["time"][0]
    end_time = all_df["time"].values[-1]
    max_altitude = np.max(all_df["altitude"])
    min_altitude = np.min(all_df["altitude"])

    features = []
    feature = {
        "type": "Feature",
        "properties": {
            "start_time": str(start_time),
            "end_time": str(end_time),
            "max_altitude": str(max_altitude),
            "min_altitude": str(min_altitude),
            "total_distance": str(total_distance),
        },
        "geometry": {
            "type": "LineString",
            "coordinates": xy,
        },
    }

    features.append(feature)

    # GeoJSONファイルを書き込む
    with open(ofile, "w", encoding='utf_8') as f:
        json.dump({"type":"FeatureCollection", "features": features}, f, indent = 2, ensure_ascii=False)